In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import pymysql

csv를 DataFrame으로 변환

In [2]:
wt_df = pd.read_csv('./data/wt210101_241030.csv', encoding='cp949')

In [3]:
print( wt_df.info() )
print( wt_df.head(3) )
wt_df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1399 entries, 0 to 1398
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   지점          1399 non-null   int64  
 1   지점명         1399 non-null   object 
 2   일시          1399 non-null   object 
 3   평균기온(°C)    1399 non-null   float64
 4   최저기온(°C)    1398 non-null   float64
 5   최고기온(°C)    1399 non-null   float64
 6   일강수량(mm)    591 non-null    float64
 7   평균 풍속(m/s)  1394 non-null   float64
 8   일 최심적설(cm)  92 non-null     float64
dtypes: float64(6), int64(1), object(2)
memory usage: 98.5+ KB
None
    지점 지점명          일시  평균기온(°C)  최저기온(°C)  최고기온(°C)  일강수량(mm)  평균 풍속(m/s)  \
0  108  서울  2021-01-01      -4.2      -9.8       1.6       NaN         2.0   
1  108  서울  2021-01-02      -5.0      -8.4      -1.4       NaN         2.6   
2  108  서울  2021-01-03      -5.6      -9.1      -2.0       NaN         2.0   

   일 최심적설(cm)  
0         NaN  
1         NaN  
2         NaN 

Index(['지점', '지점명', '일시', '평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '일강수량(mm)',
       '평균 풍속(m/s)', '일 최심적설(cm)'],
      dtype='object')

# db저장

In [4]:
conn = pymysql.connect(host="localhost", port=3306, db="mp",
                       user="root", passwd="1234")

# 2. 명령 도구 준비
cursor = conn.cursor()

In [5]:
# 테이블 생성 쿼리
create_table_query = """
CREATE TABLE IF NOT EXISTS weather_data (
    station_id INT,
    station_name VARCHAR(50),
    date_time varchar(30),
    avg_temp FLOAT,
    min_temp FLOAT,
    max_temp FLOAT,
    daily_precipitation FLOAT,
    avg_wind_speed FLOAT,
    daily_snow_depth FLOAT
)
"""
cursor.execute(create_table_query)

0

In [6]:
# DataFrame의 각 행을 MySQL에 삽입
for _, row in wt_df.iterrows():
    insert_query = """
    INSERT INTO weather_data (
        station_id, station_name, date_time,
        avg_temp, min_temp, max_temp,
        daily_precipitation, avg_wind_speed, daily_snow_depth
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    # np.nan을 None으로 변환하는 함수
    def convert_nan(value):
        return None if isinstance(value, float) and np.isnan(value) else value
    
    values = (
        convert_nan(row['지점']),
        convert_nan(row['지점명']),
        convert_nan(row['일시']),
        convert_nan(row['평균기온(°C)']),
        convert_nan(row['최저기온(°C)']),
        convert_nan(row['최고기온(°C)']),
        convert_nan(row['일강수량(mm)']),
        convert_nan(row['평균 풍속(m/s)']),
        convert_nan(row['일 최심적설(cm)'])
    )
    cursor.execute(insert_query, values)

In [ ]:
# 변경사항 저장
conn.commit()

In [8]:
conn.close()
cursor.close()

In [ ]:
# db 저장 완료